In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import simweights
import pickle
import os, sys
import numpy as np
import matplotlib as mat
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.colors as colors
import matplotlib.gridspec as gridspec
import pandas as pd
import tables
import h5py
import math
from scipy.stats import mstats
import matplotlib as mpl
import matplotlib.font_manager as font_manager


In [3]:
file_path = "/data/user/tvaneede/GlobalFit/reco_processing/hdf/output/ftp_l3casc/NuTau_22634_0000000-0000999.h5"
hdf = pd.HDFStore(file_path,'r')
nfiles = 1000

In [4]:
weighter = simweights.NuGenWeighter(hdf, nfiles=nfiles)

In [5]:
# astro
per_flavor_norm = 2.12
gamma_astro = 2.87
livetime_yr = 11.687
livetime_s  = livetime_yr * 365.25 * 24 * 3600 # 11.687 year
def AstroFluxModel(pdgid, energy, cos_zen):
    flux = 0.5*(per_flavor_norm*1e-18)*(energy/1e5)**-gamma_astro
    return flux
weights  = weighter.get_weights( AstroFluxModel )

In [6]:
# variables
PrimaryNeutrinoEnergy = hdf["I3MCWeightDict"]["PrimaryNeutrinoEnergy"].values
HESE_CausalQTot = hdf["HESE_CausalQTot"].value.values
HESE_VHESelfVeto = hdf["HESE_VHESelfVeto"].value.values
QFilterMask_HESEFilter_15_condition_passed = hdf["QFilterMask_HESEFilter_15_condition_passed"].value.values

In [8]:
# reproduce event rate
mask = (HESE_VHESelfVeto == False) & (HESE_CausalQTot > 6000)
weights_cut = weights[mask]
print("weights_cut", sum(weights_cut)*livetime_s)

# now with QFilterMask
mask = (QFilterMask_HESEFilter_15_condition_passed == 1)
weights_cut = weights[mask]
print("weights_cut", sum(weights_cut)*livetime_s)
print("total events", len(weights_cut))

# now with QFilterMask and Qtot
mask = (QFilterMask_HESEFilter_15_condition_passed == 1)  & (HESE_CausalQTot > 6000)
weights_cut = weights[mask]
print("weights_cut", sum(weights_cut)*livetime_s)
print("total events", len(weights_cut))


weights_cut 32.41837213952271
weights_cut 175.9312620116005
total events 27352
weights_cut 32.41837213952271
total events 10754


In [9]:
file_path_filtered = "/data/user/tvaneede/GlobalFit/reco_processing/hdf/output/v1_wpid/NuTau_22634_0000000-0000999.h5"
hdf_filtered = pd.HDFStore(file_path_filtered,'r')
print("total filtered events", len(hdf_filtered["I3MCWeightDict"]["PrimaryNeutrinoEnergy"].values))

total filtered events 14262


Why do I miss so many filtered events with my 

tray.Add( lambda frame: 'QFilterMask' in frame and frame['QFilterMask']['HESEFilter_15'].condition_passed )

In [ ]:
# events in my filtered file
mask = hdf_filtered["I3EventHeader"]["Run"] == 2263400000
print(hdf_filtered["I3EventHeader"][mask])

# actual events
mask = (hdf["I3EventHeader"]["Run"] == 2263400000) & (QFilterMask_HESEFilter_15_condition_passed == 1)
print(hdf["I3EventHeader"][mask])


            Run  Event  SubEvent  SubEventStream  exists  time_start_utc_daq  \
986  2263400000     88         0               0       1  130612472805721229   
987  2263400000    100         0               0       1  130612472805776339   
988  2263400000    605         0               0       1  130612473106104099   
989  2263400000    654         0               0       1  130612472807102499   

     time_start_mjd    time_end_utc_daq  time_end_mjd  
986    59000.171844  130612472805917299  59000.171844  
987    59000.171844  130612472805989439  59000.171844  
988    59000.171844  130612473106305169  59000.171844  
989    59000.171844  130612472807305009  59000.171844  
               Run  Event  SubEvent  SubEventStream  exists  \
141121  2263400000     88         0               0       1   
141122  2263400000    100         0               0       1   
141128  2263400000    605         0               0       1   
141131  2263400000    654         0               0       1   
1411

Now lets take a look in the l3 file

dataio-shovel /data/sim/IceCube/2023/filtered/level3/cascade/neutrino-generator/22634/0000000-0000999/Level3_NuTau_NuGenCCNC.022634.000000.i3.zst

In there, we have event 862 that has QFilter passed.

Now look in our filtered file.

dataio-shovel /data/user/tvaneede/GlobalFit/reco_processing/filtering/output/v1/22634/0000000-0000999/Level3_NuTau_NuGenCCNC.022634.000000.i3.zst

There we also see the event!

Lets see my reco file

dataio-shovel /data/user/tvaneede/GlobalFit/reco_processing/output/v1/22634/0000000-0000999/Reco_NuTau_NuGenCCNC.022634.000000.i3.zst_out.i3.bz2 

There we also have the event!

Let's see the PID file

dataio-shovel /data/user/tvaneede/GlobalFit/reco_processing/pid_neha/output/v1/22634/0000000-0000999/Reco_NuTau_NuGenCCNC.022634.000000.i3.zst_out.i3.bz2

There we suddenly have 4 events left! Wtf happenend.

In [ ]:
# now lets take a look in 